# Orange Hi-Tech Girls: AI

## **Zadanie**

W Orange wykorzystujemy wiele różnych urządzeń tworzących infrastrukturę naszej sieci. Znajdują się one w różnych lokalizacjach, np. w budynkach lub na budynkach, w różnych miejscach w przestrzeni publicznej. Na ich działanie może wpływać wiele czynników – jednym z nich jest pogoda.

Twoim zadaniem jest przeanalizowanie pogody (np. temperatury, opadów, siły wiatru) w lokalizacjach, w których Orange oraz konkurencyjni operatorzy posiadają stacje BTS, biorąc pod uwagę także awaryjność BTS Orange w kolejnych miesiącach lat 2017 i 2022.

## **Możesz wykorzystać dane ze źródeł**

*   dane pogodowe zawarte w plikach „A..”, „B…” itd. – wartości wskaźników temperatury w stacjach w kolejnych dniach lat 2017 i 2022, gdzie H1 i H2 oznacza pierwszą i drugą połowę roku (informacja o lokalizacji stacji znajduje się w pliku „kody_stacji.csv”, a wyjaśnienie kodów parametrów – w pliku „kody_parametr.csv”,
*   dane dot. BTS w plikach „bts.csv” (operator, lokalizacja, technologia obsługiwana przez stację),
*   dane dot. awaryjności BTS w pliku (bts_anomalie_pct.xlsx) – odsetek anomalii dot. funkcjonowania BTS w danym miesiącu (im wyższy odsetek, tym gorzej funkcjonuje BTS).

Dane są sztucznie wygenerowane i nie przedstawiają rzeczywistej sytuacji Orange ani konkurencji.

[Plik do pobrania](https://hitechgirls.orange.pl/files/resources/Zadanie_dataai.zip)

## **Przygotuj zbiór danych**

*   połączenie źródeł danych w jeden dataset za pomocą odpowiednich id,
*   agregacja danych pogodowych do większych częstotliwości np. średnie dobowe czy miesięczne,
*   połączenie danych dot. lokalizacji łącząc współrzędne geograficzne z nazwami miejscowości.

## **Przeprowadź następującą analizę danych**

*   analiza zbiorów danych – statystyki, podsumowania, analizę danych,
*   zidentyfikowanie stacji, które są narażone na najbardziej ekstremalne warunki pogodowe/największe zmiany pogody w badanym czasie,
*   wizualizacja danych pogodowych i lokalizacji sieci na mapie,
*   porównanie sieci Orange z innymi sieciami,
*   ocena wpływu pogody na awaryjność BTS.

## **Dodatkowo możesz wytrenować model ML**

*   clustering – identyfikacja stacji o podobnych warunkach pogodowych i awaryjności,
*   model klasyfikacyjny przewidujący wystąpienie anomalii powyżej zadanego progu,
*   model przewidujący jaki odsetek anomalii wystąpi w danym miesiącu w określonej lokalizacji.

Nie musisz realizować wszystkich punktów analizy ani korzystać ze wszystkich danych. Do Ciebie należy decyzja czy chcesz się skupić na danych jednego operatora czy wielu, czy użyjesz wskaźników pogodowych, a jeśli tak to których, czy porównasz ze sobą dane z 2017 i 2022, czy wykorzystasz tylko jeden rok albo krótszy okres.

*Analizę danych wykonaj w języku Python, z wykorzystaniem wszystkich pakietów, jakich potrzebujesz. Swoje rozwiązanie dostarcz w postaci Jupyter notebook w dwóch formatach: ipynb oraz .html (możesz wygenerować .html za pomocą opcji "Zapisz jako"/"Download as").*

## **Rozwiązanie powinno zawierać**

*   całość kodu jaką stworzyłaś na potrzeby zadania,
*   opis metodologii, jaką zastosowałaś - np. krótkie komentarze krok po kroku, co wykonujesz w kodzie.
*   wnioski z analizy, opisy wizualizacji,
*   krótki opis Twoich pomysłów na rozwój rozwiązania i jego zastosowanie w praktyce.

Do opisów możesz użyć Markdowna albo zamieścić je jako komentarze w kodzie.

# Rozwiązanie

## Przygotowanie

### Import bibliotek

In [1]:
'''
# pobranie sweetviz
!pip install sweetviz -qq
!echo '[Layout]' > Override.ini
!echo 'show_logo = 0' >> Override.ini


import sweetviz as sv

sv.config_parser.read("Override.ini")
my_report = sv.analyze(data_raw)
my_report.show_html(filepath='Basic_report.html', layout='vertical', scale=None)
my_report.show_notebook()
'''

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 83.6 MB/s eta 0:00:00


In [2]:
# importowanie potrzebnych bibliotek
import pandas as pd

In [3]:
# sprawdzenie wersji pandas
pd.__version__

'1.5.3'

## Parametry - łączenie danych

### Parametr A

In [5]:
# wczytanie pliku A_2017_H1.csv
A_2017_H1 = pd.read_csv('https://drive.google.com/uc?id=1PHN7ENUUyHKRCM356wLp1DAVUHc4tX0E', encoding='cp1250', sep=';')

In [6]:
A_2017_H1

,ID_stacji,Parametr,Data_godzina,Wartość
0,538,A,01.01.2017 00:00,"7,5"
1,538,A,01.01.2017 01:00,"8,7"
2,538,A,01.01.2017 02:00,"8,7"
3,538,A,01.01.2017 03:00,"9,3"
4,538,A,01.01.2017 04:00,"8,4"
...,...,...,...,...
1019584,560,A,30.06.2017 19:00,"25,9"
1019585,560,A,30.06.2017 20:00,"22,5"
1019586,560,A,30.06.2017 21:00,"22,6"
1019587,560,A,30.06.2017 22:00,"22,5"


In [7]:
# wczytanie pliku A_2017_H2.csv
A_2017_H2 = pd.read_csv('https://drive.google.com/uc?id=1Aill8jh9D2EBHh0pvUloB-FAOnsyxhU4', encoding='cp1250', sep=';')

In [8]:
A_2017_H2

,ID_stacji,Parametr,Data_godzina,Wartość
0,538,A,01.07.2017 00:00,"16,2"
1,538,A,01.07.2017 01:00,"14,8"
2,538,A,01.07.2017 02:00,"15,6"
3,538,A,01.07.2017 03:00,"13,4"
4,538,A,01.07.2017 04:00,"15,9"
...,...,...,...,...
1029299,560,A,31.12.2017 19:00,"14,1"
1029300,560,A,31.12.2017 20:00,"12,8"
1029301,560,A,31.12.2017 21:00,"14,9"
1029302,560,A,31.12.2017 22:00,"13,8"


In [9]:
# wczytanie pliku A_2022_H1.csv
A_2022_H1 = pd.read_csv('https://drive.google.com/uc?id=1N3y8t19_m-cusCgMabCbp0lUVPuDep48', encoding='cp1250', sep=';')

In [10]:
A_2022_H1

,ID_stacji,Parametr,Data_godzina,Wartość
0,538,A,01.01.2022 00:00,"7,9"
1,538,A,01.01.2022 01:00,"8,6"
2,538,A,01.01.2022 02:00,"8,7"
3,538,A,01.01.2022 03:00,"9,1"
4,538,A,01.01.2022 04:00,"8,5"
...,...,...,...,...
1019584,560,A,30.06.2022 19:00,"25,6"
1019585,560,A,30.06.2022 20:00,23
1019586,560,A,30.06.2022 21:00,"22,9"
1019587,560,A,30.06.2022 22:00,"22,7"


In [11]:
# wczytanie pliku A_2022_H2.csv
A_2022_H2 = pd.read_csv('https://drive.google.com/uc?id=1ps_QGu3QT-ePS13By6myDgwRcT0Nh7Kg', encoding='cp1250', sep=';')

In [12]:
A_2022_H2

,ID_stacji,Parametr,Data_godzina,Wartość
0,538,A,01.07.2022 00:00,"16,6"
1,538,A,01.07.2022 01:00,15
2,538,A,01.07.2022 02:00,"15,2"
3,538,A,01.07.2022 03:00,"13,9"
4,538,A,01.07.2022 04:00,"16,1"
...,...,...,...,...
1029299,560,A,31.12.2022 19:00,"13,6"
1029300,560,A,31.12.2022 20:00,"12,9"
1029301,560,A,31.12.2022 21:00,"14,7"
1029302,560,A,31.12.2022 22:00,"13,6"


In [13]:
par_A = pd.concat([A_2017_H1, A_2017_H2, A_2022_H1, A_2022_H2], axis=0, ignore_index=True)

In [14]:
par_A

,ID_stacji,Parametr,Data_godzina,Wartość
0,538,A,01.01.2017 00:00,"7,5"
1,538,A,01.01.2017 01:00,"8,7"
2,538,A,01.01.2017 02:00,"8,7"
3,538,A,01.01.2017 03:00,"9,3"
4,538,A,01.01.2017 04:00,"8,4"
...,...,...,...,...
4097781,560,A,31.12.2022 19:00,"13,6"
4097782,560,A,31.12.2022 20:00,"12,9"
4097783,560,A,31.12.2022 21:00,"14,7"
4097784,560,A,31.12.2022 22:00,"13,6"


### Parametr B

In [15]:
# wczytanie pliku B_2017_H1.csv
B_2017_H1 = pd.read_csv('https://drive.google.com/uc?id=1D_ctClz_TPWPqqzGdJjmCAFER0W9fieD', encoding='cp1250', sep=';')

In [16]:
B_2017_H1

,ID_stacji,Parametr,Data_godzina,Wartość
0,445,B,01.01.2017 00:00,"15,5"
1,445,B,01.01.2017 01:00,"14,4"
2,445,B,01.01.2017 02:00,"15,5"
3,445,B,01.01.2017 03:00,"15,0"
4,445,B,01.01.2017 04:00,"15,3"
...,...,...,...,...
1021416,556,B,30.06.2017 19:00,"20,5"
1021417,556,B,30.06.2017 20:00,"17,6"
1021418,556,B,30.06.2017 21:00,"15,3"
1021419,556,B,30.06.2017 22:00,"15,6"


In [17]:
# wczytanie pliku B_2017_H2.csv
B_2017_H2 = pd.read_csv('https://drive.google.com/uc?id=1e7BDPc-E8gXjP0kcvO10fbTro2MOPgXK', encoding='cp1250', sep=';')

In [18]:
B_2017_H2

,ID_stacji,Parametr,Data_godzina,Wartość
0,445,B,01.07.2017 00:00,"15,7"
1,445,B,01.07.2017 01:00,"14,6"
2,445,B,01.07.2017 02:00,14
3,445,B,01.07.2017 03:00,"13,9"
4,445,B,01.07.2017 04:00,22
...,...,...,...,...
1029604,556,B,31.12.2017 19:00,"14,2"
1029605,556,B,31.12.2017 20:00,"14,9"
1029606,556,B,31.12.2017 21:00,"15,1"
1029607,556,B,31.12.2017 22:00,"14,5"


In [19]:
# wczytanie pliku B_2022_H1.csv
B_2022_H1 = pd.read_csv('https://drive.google.com/uc?id=1meCyNosCBZiSeESBXI-3_bx2WL49ioBQ', encoding='cp1250', sep=';')

In [20]:
B_2022_H1

,ID_stacji,Parametr,Data_godzina,Wartość
0,445,B,01.01.2022 00:00,"14,6"
1,445,B,01.01.2022 01:00,"14,4"
2,445,B,01.01.2022 02:00,"14,8"
3,445,B,01.01.2022 03:00,"14,1"
4,445,B,01.01.2022 04:00,"14,6"
...,...,...,...,...
1021416,556,B,30.06.2022 19:00,"19,8"
1021417,556,B,30.06.2022 20:00,"16,8"
1021418,556,B,30.06.2022 21:00,"15,1"
1021419,556,B,30.06.2022 22:00,"14,8"


In [21]:
# wczytanie pliku B_2022_H2.csv
B_2022_H2 = pd.read_csv('https://drive.google.com/uc?id=12qOrZL74pQWg7uFdsYCeLxtVe2lxPK3M', encoding='cp1250', sep=';')

In [22]:
B_2022_H2

,ID_stacji,Parametr,Data_godzina,Wartość
0,445,B,01.07.2022 00:00,"15,9"
1,445,B,01.07.2022 01:00,"15,5"
2,445,B,01.07.2022 02:00,"14,8"
3,445,B,01.07.2022 03:00,"14,6"
4,445,B,01.07.2022 04:00,"22,4"
...,...,...,...,...
1029604,556,B,31.12.2022 19:00,"14,8"
1029605,556,B,31.12.2022 20:00,15
1029606,556,B,31.12.2022 21:00,"15,1"
1029607,556,B,31.12.2022 22:00,"14,9"


In [23]:
par_B = pd.concat([B_2017_H1, B_2017_H2, B_2022_H1, B_2022_H2], axis=0, ignore_index=True)

In [24]:
par_B

,ID_stacji,Parametr,Data_godzina,Wartość
0,445,B,01.01.2017 00:00,"15,5"
1,445,B,01.01.2017 01:00,"14,4"
2,445,B,01.01.2017 02:00,"15,5"
3,445,B,01.01.2017 03:00,"15,0"
4,445,B,01.01.2017 04:00,"15,3"
...,...,...,...,...
4102055,556,B,31.12.2022 19:00,"14,8"
4102056,556,B,31.12.2022 20:00,15
4102057,556,B,31.12.2022 21:00,"15,1"
4102058,556,B,31.12.2022 22:00,"14,9"


### Parametr C

In [25]:
# wczytanie pliku C_2017_H1.csv
C_2017_H1 = pd.read_csv('https://drive.google.com/uc?id=1B0-FAPsJooigjA0bxUPJPyFN1UIqUvfF', encoding='cp1250', sep=';')

In [26]:
C_2017_H1

,ID_stacji,Parametr,Data_godzina,Wartość
0,2,C,01.01.2017 00:00,"6,1"
1,2,C,01.01.2017 01:00,"8,4"
2,2,C,01.01.2017 02:00,"7,1"
3,2,C,01.01.2017 03:00,6
4,2,C,01.01.2017 04:00,"5,8"
...,...,...,...,...
1035968,560,C,30.06.2017 19:00,"1,2"
1035969,560,C,30.06.2017 20:00,"0,8"
1035970,560,C,30.06.2017 21:00,1
1035971,560,C,30.06.2017 22:00,"1,9"


In [27]:
# wczytanie pliku C_2017_H2.csv
C_2017_H2 = pd.read_csv('https://drive.google.com/uc?id=1SZPzcPwstbCKTnkCOan5YqsZ1rblaUAl', encoding='cp1250', sep=';')

In [28]:
C_2017_H2

,ID_stacji,Parametr,Data_godzina,Wartość
0,2,C,01.07.2017 00:00,"2,1"
1,2,C,01.07.2017 01:00,"1,8"
2,2,C,01.07.2017 02:00,"2,5"
3,2,C,01.07.2017 03:00,"1,8"
4,2,C,01.07.2017 04:00,"2,6"
...,...,...,...,...
1074974,349,C,10.12.2017 09:00,4
1074975,349,C,10.12.2017 10:00,"4,7"
1074976,349,C,10.12.2017 11:00,"3,8"
1074977,349,C,10.12.2017 12:00,"3,7"


In [29]:
# wczytanie pliku C_2022_H1.csv
C_2022_H1 = pd.read_csv('https://drive.google.com/uc?id=1E9BYBWESGeeGoowULUFJbROt2h3Vajve', encoding='cp1250', sep=';')

In [30]:
C_2022_H1

,ID_stacji,Parametr,Data_godzina,Wartość
0,2,C,01.01.2022 00:00,"6,7"
1,2,C,01.01.2022 01:00,"8,3"
2,2,C,01.01.2022 02:00,"7,9"
3,2,C,01.01.2022 03:00,"5,6"
4,2,C,01.01.2022 04:00,"5,7"
...,...,...,...,...
1035968,560,C,30.06.2022 19:00,"1,5"
1035969,560,C,30.06.2022 20:00,"0,9"
1035970,560,C,30.06.2022 21:00,"1,2"
1035971,560,C,30.06.2022 22:00,"1,6"


In [31]:
# wczytanie pliku C_2022_H2.csv
C_2022_H2 = pd.read_csv('https://drive.google.com/uc?id=1eCDBsycTi-Vew2Zg43nAiFclL4RwOSiv', encoding='cp1250', sep=';')

In [32]:
C_2022_H2

,ID_stacji,Parametr,Data_godzina,Wartość
0,2,C,01.07.2022 00:00,"1,5"
1,2,C,01.07.2022 01:00,"2,3"
2,2,C,01.07.2022 02:00,"2,8"
3,2,C,01.07.2022 03:00,"2,5"
4,2,C,01.07.2022 04:00,2
...,...,...,...,...
1074974,349,C,10.12.2022 09:00,"4,5"
1074975,349,C,10.12.2022 10:00,"4,3"
1074976,349,C,10.12.2022 11:00,"4,3"
1074977,349,C,10.12.2022 12:00,"4,1"


In [33]:
par_C = pd.concat([C_2017_H1, C_2017_H2, C_2022_H1, C_2022_H2], axis=0, ignore_index=True)

In [34]:
par_C

,ID_stacji,Parametr,Data_godzina,Wartość
0,2,C,01.01.2017 00:00,"6,1"
1,2,C,01.01.2017 01:00,"8,4"
2,2,C,01.01.2017 02:00,"7,1"
3,2,C,01.01.2017 03:00,6
4,2,C,01.01.2017 04:00,"5,8"
...,...,...,...,...
4221899,349,C,10.12.2022 09:00,"4,5"
4221900,349,C,10.12.2022 10:00,"4,3"
4221901,349,C,10.12.2022 11:00,"4,3"
4221902,349,C,10.12.2022 12:00,"4,1"


### Parametr D

In [35]:
# wczytanie pliku D_2017.csv
D_2017 = pd.read_csv('https://drive.google.com/uc?id=1usXqOugY4f5HkCDSpdbFsUmZWApSK6JO', encoding='cp1250', sep=';')

<ipython-input-35-22af4b5c08b6>:2: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  D_2017 = pd.read_csv('https://drive.google.com/uc?id=1usXqOugY4f5HkCDSpdbFsUmZWApSK6JO', encoding='cp1250', sep=';')


In [36]:
D_2017

,ID_stacji,Parametr,Data_godzina,Wartość
0,1.0,D,01.01.2017 06:00,"3,7"
1,1.0,D,02.01.2017 06:00,4
2,1.0,D,03.01.2017 06:00,"0,9"
3,1.0,D,04.01.2017 06:00,"5,5"
4,1.0,D,05.01.2017 06:00,"1,7"
...,...,...,...,...
1035968,NaN,NaN,NaN,NaN
1035969,NaN,NaN,NaN,NaN
1035970,NaN,NaN,NaN,NaN
1035971,NaN,NaN,NaN,NaN


In [37]:
# wczytanie pliku D_2022.csv
D_2022 = pd.read_csv('https://drive.google.com/uc?id=12TC9WMzvxkl0_DCRNQduWU28GAsyCCUX', encoding='cp1250', sep=';')

<ipython-input-37-4bc1a6d89998>:2: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  D_2022 = pd.read_csv('https://drive.google.com/uc?id=12TC9WMzvxkl0_DCRNQduWU28GAsyCCUX', encoding='cp1250', sep=';')


In [38]:
D_2022

,ID_stacji,Parametr,Data_godzina,Wartość
0,1.0,D,01.01.2022 06:00,"3,5"
1,1.0,D,02.01.2022 06:00,4
2,1.0,D,03.01.2022 06:00,"1,2"
3,1.0,D,04.01.2022 06:00,"6,4"
4,1.0,D,05.01.2022 06:00,"1,4"
...,...,...,...,...
1035968,NaN,NaN,NaN,NaN
1035969,NaN,NaN,NaN,NaN
1035970,NaN,NaN,NaN,NaN
1035971,NaN,NaN,NaN,NaN


In [39]:
par_D = pd.concat([D_2017, D_2022], axis=0, ignore_index=True)

In [40]:
par_D

,ID_stacji,Parametr,Data_godzina,Wartość
0,1.0,D,01.01.2017 06:00,"3,7"
1,1.0,D,02.01.2017 06:00,4
2,1.0,D,03.01.2017 06:00,"0,9"
3,1.0,D,04.01.2017 06:00,"5,5"
4,1.0,D,05.01.2017 06:00,"1,7"
...,...,...,...,...
2071941,NaN,NaN,NaN,NaN
2071942,NaN,NaN,NaN,NaN
2071943,NaN,NaN,NaN,NaN
2071944,NaN,NaN,NaN,NaN


### Parametr E

In [41]:
# wczytanie pliku E_2017_H1.csv
E_2017_H1 = pd.read_csv('https://drive.google.com/uc?id=1Bm9AA8hieaTDExz8DcwlK6TVMI06nd2b', encoding='cp1250', sep=';')

In [42]:
E_2017_H1

,ID_stacji,Parametr,Data_godzina,Wartość
0,2,E,01.01.2017 00:00,95
1,2,E,01.01.2017 01:00,99
2,2,E,01.01.2017 02:00,96
3,2,E,01.01.2017 03:00,96
4,2,E,01.01.2017 04:00,93
...,...,...,...,...
1036138,560,E,30.06.2017 19:00,66
1036139,560,E,30.06.2017 20:00,84
1036140,560,E,30.06.2017 21:00,75
1036141,560,E,30.06.2017 22:00,91


In [43]:
# wczytanie pliku E_2017_H2.csv
E_2017_H2 = pd.read_csv('https://drive.google.com/uc?id=1sSBzCGKmfduY67irUtW5ARTYiGLWs7-_', encoding='cp1250', sep=';')

In [44]:
E_2017_H2

,ID_stacji,Parametr,Data_godzina,Wartość
0,2,E,01.07.2017 00:00,94
1,2,E,01.07.2017 01:00,93
2,2,E,01.07.2017 02:00,97
3,2,E,01.07.2017 03:00,97
4,2,E,01.07.2017 04:00,103
...,...,...,...,...
1047929,554,E,12.12.2017 12:00,86
1047930,554,E,12.12.2017 13:00,82
1047931,554,E,12.12.2017 14:00,75
1047932,554,E,12.12.2017 15:00,79


In [45]:
# wczytanie pliku E_2022_H1.csv
E_2022_H1 = pd.read_csv('https://drive.google.com/uc?id=1FX3dTG432RKZVQc7L5kz9G-Ib8Timz-4', encoding='cp1250', sep=';')

In [46]:
E_2022_H1

,ID_stacji,Parametr,Data_godzina,Wartość
0,2,E,01.01.2022 00:00,97
1,2,E,01.01.2022 01:00,97
2,2,E,01.01.2022 02:00,95
3,2,E,01.01.2022 03:00,95
4,2,E,01.01.2022 04:00,97
...,...,...,...,...
1036138,560,E,30.06.2022 19:00,68
1036139,560,E,30.06.2022 20:00,87
1036140,560,E,30.06.2022 21:00,80
1036141,560,E,30.06.2022 22:00,89


In [47]:
# wczytanie pliku E_2022_H2.csv
E_2022_H2 = pd.read_csv('https://drive.google.com/uc?id=1Gcarx8pUj5hgjK3kI8j0R3Et3WI7jta6', encoding='cp1250', sep=';')

In [48]:
E_2022_H2

,ID_stacji,Parametr,Data_godzina,Wartość
0,2,E,01.07.2022 00:00,96
1,2,E,01.07.2022 01:00,98
2,2,E,01.07.2022 02:00,96
3,2,E,01.07.2022 03:00,102
4,2,E,01.07.2022 04:00,98
...,...,...,...,...
1047929,554,E,12.12.2022 12:00,87
1047930,554,E,12.12.2022 13:00,79
1047931,554,E,12.12.2022 14:00,77
1047932,554,E,12.12.2022 15:00,74


In [49]:
par_E = pd.concat([E_2017_H1, E_2017_H2, E_2022_H1, E_2022_H2], axis=0, ignore_index=True)

In [50]:
par_E

,ID_stacji,Parametr,Data_godzina,Wartość
0,2,E,01.01.2017 00:00,95
1,2,E,01.01.2017 01:00,99
2,2,E,01.01.2017 02:00,96
3,2,E,01.01.2017 03:00,96
4,2,E,01.01.2017 04:00,93
...,...,...,...,...
4168149,554,E,12.12.2022 12:00,87
4168150,554,E,12.12.2022 13:00,79
4168151,554,E,12.12.2022 14:00,77
4168152,554,E,12.12.2022 15:00,74


### Czyszczenie danych

In [51]:
# sprawdzanie obecności wartości NaN wśród parametrów
print('\nNaN par_A:', par_A.isna().values.any(),
      '\nNaN par_B:', par_B.isna().values.any(),
      '\nNaN par_C:', par_C.isna().values.any(),
      '\nNaN par_D:', par_D.isna().values.any(),
      '\nNaN par_E:', par_E.isna().values.any())


NaN par_A: False 
NaN par_B: False 
NaN par_C: False 
NaN par_D: True 
NaN par_E: False


In [52]:
# usuwanie wartości NaN w tabeli par_D
par_D.dropna(how='all', inplace=True)
print('Nan par_D:', par_D.isna().values.any())

Nan par_D: False


### Połączenie parametrów

In [53]:
par = pd.concat([par_A, par_B, par_C, par_D, par_E], axis=0, ignore_index=True)

In [54]:
par

,ID_stacji,Parametr,Data_godzina,Wartość
0,538.0,A,01.01.2017 00:00,"7,5"
1,538.0,A,01.01.2017 01:00,"8,7"
2,538.0,A,01.01.2017 02:00,"8,7"
3,538.0,A,01.01.2017 03:00,"9,3"
4,538.0,A,01.01.2017 04:00,"8,4"
...,...,...,...,...
16972851,554.0,E,12.12.2022 12:00,87
16972852,554.0,E,12.12.2022 13:00,79
16972853,554.0,E,12.12.2022 14:00,77
16972854,554.0,E,12.12.2022 15:00,74


In [55]:
# double-check czy są wartości NaN w złączonej tabeli
print('Czy są jakieś wartości NaN:', par.isna().values.any())

Czy są jakieś wartości NaN: False


### Obsługa typów danych

In [60]:
# sprawdzenie typów danych
par.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16972856 entries, 0 to 16972855
Data columns (total 4 columns):
 #   Column        Dtype  
---  ------        -----  
 0   ID_stacji     float64
 1   Parametr      object 
 2   Data_godzina  object 
 3   Wartość       object 
dtypes: float64(1), object(3)
memory usage: 518.0+ MB


In [101]:
# Zmiana danych
# ID -> int
par['ID_stacji'] = par['ID_stacji'].astype(int)

In [72]:
# Data_godzina -> datetime
par['Data_godzina'] = pd.to_datetime(par['Data_godzina'])

In [99]:
# Wartość -> float
par['Wartość'] = par['Wartość'].str.replace(',','.')
par['Wartość'] = par['Wartość'].astype(float)

In [102]:
par.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16972856 entries, 0 to 16972855
Data columns (total 4 columns):
 #   Column        Dtype         
---  ------        -----         
 0   ID_stacji     int64         
 1   Parametr      object        
 2   Data_godzina  datetime64[ns]
 3   Wartość       float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 518.0+ MB


## Łączenie parametrów z kodami

### Edycja kodów stacji

In [56]:
# wczytanie pliku kody_stacji.csv
kody_stacji = pd.read_csv('https://drive.google.com/uc?id=1KCedOUXTk2b3pNOvbOobS5nYmv4rFF4z', encoding='cp1250', sep=';')

In [57]:
kody_stacji

,ID,Nazwa,Szerokość geograficzna,Długość geograficzna
0,1,RYBNIK-STODOŁY,50 09 16,18 28 59
1,2,DZIWNÓW,54 01 20,14 43 54
2,3,ZIELENIEC,50 19 48,16 23 39
3,4,MIĘDZYGÓRZE,50 13 06,16 46 23
4,5,MIĘDZYLESIE,50 09 12,16 40 15
...,...,...,...,...
575,576,TUROWO,21 54 06,141
576,577,BUKOWIEC,22 24 49,435
577,578,SITOWIEC,17 45 15,134
578,579,MESZNO,17 08 27,212


In [58]:
kody_stacji.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      580 non-null    int64 
 1   Nazwa                   580 non-null    object
 2   Szerokość geograficzna  580 non-null    object
 3   Długość geograficzna    580 non-null    object
dtypes: int64(1), object(3)
memory usage: 18.2+ KB


In [141]:
kody_stacji['Szerokość geograficzna'].unique()

array(['50 09 16', '54 01 20', '50 19 48', '50 13 06', '50 09 12',
       '50 45 31', '50 28 20', '50 24 41', '50 18 08', '50 17 17',
       '50 10 55', '50 34 09', '50 25 44', '50 56 31', '50 55 19',
       '51 31 41', '54 13 24', '49 59 31', '49 51 45', '51 05 56',
       '51 02 59', '50 38 24', '51 02 30', '51 01 07', '51 46 09',
       '52 48 57', '52 45 17', '52 00 37', '52 14 34', '52 33 43',
       '52 43 39', '52 44 31', '52 24 43', '52 58 21', '52 56 15',
       '52 54 53', '52 51 44', '52 42 26', '53 50 15', '53 39 02',
       '53 25 51', '49 38 25', '49 36 01', '49 33 35', '49 32 41',
       '49 31 34', '49 30 37', '49 28 27', '49 29 43', '49 27 57',
       '49 27 54', '49 27 19', '49 26 44', '49 25 04', '49 24 44',
       '49 20 15', '49 20 59', '49 15 49', '49 14 38', '49 12 49',
       '49 59 14', '49 52 40', '49 44 07', '49 44 08', '49 39 19',
       '49 45 15', '49 45 13', '49 42 11', '49 41 00', '49 39 17',
       '49 38 41', '49 36 13', '49 35 07', '49 58 21', '49 56 

In [138]:
# stworzenie kopii df do edycji
kody = kody_stacji.copy()

In [139]:
# Zmiana wartości kolumny szerokość geograficzna
# dodanie kierunku, stopni, minut, sekund
kody[['Szer_st', 'Szer_m', 'Szer_s']] = kody['Szerokość geograficzna'].str.split(' ', expand=True)
kody.drop(columns='Szerokość geograficzna', inplace=True)
kody['Szerokość geograficzna'] = 'N ' + kody.Szer_st + '° ' + kody.Szer_m + '\' ' + kody.Szer_s + '" '
kody.drop(columns=['Szer_st', 'Szer_m', 'Szer_s'], inplace=True)
kody

,ID,Nazwa,Długość geograficzna,Szerokość geograficzna
0,1,RYBNIK-STODOŁY,18 28 59,"N 50° 09' 16"""
1,2,DZIWNÓW,14 43 54,"N 54° 01' 20"""
2,3,ZIELENIEC,16 23 39,"N 50° 19' 48"""
3,4,MIĘDZYGÓRZE,16 46 23,"N 50° 13' 06"""
4,5,MIĘDZYLESIE,16 40 15,"N 50° 09' 12"""
...,...,...,...,...
575,576,TUROWO,141,"N 21° 54' 06"""
576,577,BUKOWIEC,435,"N 22° 24' 49"""
577,578,SITOWIEC,134,"N 17° 45' 15"""
578,579,MESZNO,212,"N 17° 08' 27"""


In [140]:
# Zmiana wartości kolumny długość geograficzna
# dodanie kierunku, stopni, minut, sekund
kody[['Dł_st', 'Dł_m', 'Dł_s']] = kody['Długość geograficzna'].str.split(' ', expand=True)
kody.drop(columns='Długość geograficzna', inplace=True)
kody['Długość geograficzna'] = 'E ' + kody.Dł_st + '° ' + kody.Dł_m + '\' ' + kody.Dł_s + '" '
kody.drop(columns=['Dł_st', 'Dł_m', 'Dł_s'], inplace=True)
kody

,ID,Nazwa,Szerokość geograficzna,Długość geograficzna
0,1,RYBNIK-STODOŁY,"N 50° 09' 16""","E 18° 28' 59"""
1,2,DZIWNÓW,"N 54° 01' 20""","E 14° 43' 54"""
2,3,ZIELENIEC,"N 50° 19' 48""","E 16° 23' 39"""
3,4,MIĘDZYGÓRZE,"N 50° 13' 06""","E 16° 46' 23"""
4,5,MIĘDZYLESIE,"N 50° 09' 12""","E 16° 40' 15"""
...,...,...,...,...
575,576,TUROWO,"N 21° 54' 06""",NaN
576,577,BUKOWIEC,"N 22° 24' 49""",NaN
577,578,SITOWIEC,"N 17° 45' 15""",NaN
578,579,MESZNO,"N 17° 08' 27""",NaN


In [ ]:
'''
Dodać sprawdzenie danych długości i szerokości!

In [ ]:
'''
# zmiana nazwy kolumny ID_stacji na ID w celu łączenia tabel
par.rename(columns={'ID_stacji':'ID'}, inplace=True)

In [ ]:
'''
# sprawdzenie typów wartości ID z obu tabel
print('Wartość ID z tabeli par:', par['ID'].dtype, '\nWartość ID z tabeli kody_stacji:', kody_stacji['ID'].dtype)

Wartość ID z tabeli par: float64 
Wartość ID z tabeli kody_stacji: int64


In [ ]:
#data = pd.merge(par, kody_stacji, on='ID')

### Zmiana nazw kodów parametrów

Zastosowana by zwiększyć czytelność tabeli

In [ ]:
'''
# wczytanie pliku kody_parametr.csv
kody_param = pd.read_csv('https://drive.google.com/uc?id=1cgBnbbtkAcvhkkJ2-cvL9rd7v60p2Nbi', encoding='cp1250', sep=';')

In [ ]:
#kody_param

In [ ]:
'''
# przygotowanie słownika do zmiany nazw
di = {
    'A': 'T_pow',
    'B': 'T_gruntu',
    'C': 'Vśr_wiatru',
    'D': 'Suma_opadu_1d',
    'E': 'RH_pow' # wilgotność względna
}

In [ ]:
'''
data['Parametr'] = data['Parametr'].replace(di)
data